#### BSV step 2 for sub-volume cluter, and EDI table generation

In [1]:
import numpy as np
import pandas as pd
import glob
import os

from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture

# from matplotlib.patches import Ellipse

In [35]:
'''
return the K value and covariance_type from the best GMM model according to the BIC
'''
def get_K(feature, K_num):
    lowest_bic = np.infty
    bic = []
    n_components_range = range(1, K_num+1)
    # cv_types = ["spherical", "tied", "diag", "full"]
    cv_types = ["full"]
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = GaussianMixture(
                n_components=n_components, covariance_type=cv_type,random_state=0
            )
            gmm.fit(feature)
            bic.append(gmm.bic(feature))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmm = gmm

    bic = np.array(bic)
    
    return best_gmm.n_components, best_gmm.covariance_type

'''
For each feature, assign a K value based on GMM(BIC)
Assume each patient has N features, return a list with shape (1,N) which include K for each feature
'''
def assign_K_value2features(Patient_features, K_num):
    x, y = Patient_features.shape
    K_list = []
    cov_type_list = []
    for feature_idx in range(0, y):
        X_train = np.array(Patient_features.iloc[:, feature_idx]).reshape(-1,1)
        k, cov_type = get_K(X_train, K_num)
        K_list.append(k)
        cov_type_list.append(cov_type)

    return K_list, cov_type_list


In [36]:
def RFtable_generate(path,mode):
    df = pd.read_csv(path)
    c_name_list = df.columns.to_list()

    # Since some features are not used, we only select first order and texture features for now
    c_rest_list = ['firstorder','glcm','glrlm','glszm','gldm','ngtdm']
    rest_cname_list = []
    for feature_type in c_rest_list:
        for c_names in c_name_list:
            if feature_type in c_names:
                rest_cname_list.append(c_names)

    df_rest_origin = df.loc[:,rest_cname_list]
    if mode == 'Z-socre':
        #TODO column name 有点问题
        df_rest = pd.DataFrame(data = StandardScaler().fit_transform(df_rest_origin),columns=df_rest_origin.columns)
    else:
        df_rest = df_rest_origin
    return df_rest

In [39]:
# User parameters

# feature_dir = '/home/zhenwei/Documents/ProjectCode/image_processing/SVoxel/output_SY/RF'
# EDI_dir = '/home/zhenwei/Documents/ProjectCode/image_processing/SVoxel/output_SY/EDI'

feature_dir = '/home/zhenwei/Documents/ProjectCode/image_processing/SVoxel/output_HN/RF'
EDI_dir = '/home/zhenwei/Documents/ProjectCode/image_processing/SVoxel/output_HN/EDI'

if not os.path.exists(EDI_dir):
    os.makedirs(EDI_dir)

center = 'HN'
K_num = 5   # EDI range 1-5, can be set 10 as well

Patients_K = pd.DataFrame()
Patients_covtype = []

files = glob.glob(os.path.join(feature_dir,'*.csv'))
ptid_list = []
ptid_nan = []
ptid_K0 = []

for i in range(len(files)):
    ptid = files[i].split('_')[-2].split('/')[-1]
    df_feature =pd.read_csv(files[i])
    if df_feature.isnull().values.any():
        ptid_nan.append(files[i]) # record table has NaN
    else:
        ptid = files[i].split('_')[-2].split('/')[-1]
        ptid_list.append(ptid)
        print('processing: ',ptid)

        # TODO not sure if normalization is useful for discrete numbers
        # df_rest = RFtable_generate(files[i],'Z-socre')
        df_rest = RFtable_generate(files[i],'')

        K, cov_type_list = assign_K_value2features(df_rest, K_num)
        try:
            if sum(K) !=0:
                K_new = [ptid] + K
                tmp_df_K = pd.DataFrame(data = K_new).T
                tmp_df_K.columns = ['ID']+ df_rest.columns.to_list()
                Patients_K = pd.concat([Patients_K,tmp_df_K])
            else:
                ptid_K0.append(ptid)
                print('something wrong: K is 0')
        except:
            print('Error: someting wrong about K')
            # cov types are important, but not investigated for now
        Patients_covtype.append(cov_type_list)
        print('Patient-',files[i].split('_')[-2].split('/')[-1], ' done!')

# save table in a specified directory
Patients_K.to_csv(os.path.join(EDI_dir,center+'_BSV_EDI_k5.csv'),index = False)

processing:  644272
Patient- 644272  done!
processing:  661734
Patient- 661734  done!
processing:  692817
Patient- 692817  done!
processing:  643436
Patient- 643436  done!
processing:  653439
Patient- 653439  done!
processing:  653141
Patient- 653141  done!
processing:  651041
Patient- 651041  done!
processing:  650415
Patient- 650415  done!
processing:  683718
Patient- 683718  done!
processing:  670164
Patient- 670164  done!
processing:  635536
Patient- 635536  done!
processing:  650757
Patient- 650757  done!
processing:  650605
Patient- 650605  done!
processing:  680100
Patient- 680100  done!
processing:  639805
Patient- 639805  done!
processing:  680757
Patient- 680757  done!
processing:  648401
Patient- 648401  done!
processing:  676249
Patient- 676249  done!
processing:  684239
Patient- 684239  done!
processing:  692870
Patient- 692870  done!
processing:  685498
Patient- 685498  done!
processing:  667566
Patient- 667566  done!
processing:  648729
Patient- 648729  done!
processing:

/home/zhenwei/anaconda3/envs/mma/lib/python3.8/site-packages/sklearn/mixture/_base.py:143: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/zhenwei/anaconda3/envs/mma/lib/python3.8/site-packages/sklearn/mixture/_base.py:143: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/zhenwei/anaconda3/envs/mma/lib/python3.8/site-packages/sklearn/mixture/_base.py:143: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/zhenwei/anaconda3/envs/mma/lib/python3.8/site-packages/sklearn/mixture/_base.py:143: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/zhenwei/anaconda3/envs/mma/lib/python3.8/site-packages

Patient- 643289  done!
processing:  655437
Patient- 655437  done!
processing:  688729
Patient- 688729  done!
processing:  711623
Patient- 711623  done!
processing:  651748
Patient- 651748  done!
processing:  687737
Patient- 687737  done!
processing:  659962
Patient- 659962  done!
processing:  666969
Patient- 666969  done!
processing:  638552
Patient- 638552  done!
processing:  630755
Patient- 630755  done!
processing:  640879
Patient- 640879  done!
processing:  683822
Patient- 683822  done!
processing:  692734
Patient- 692734  done!
processing:  675929
Patient- 675929  done!
processing:  633342
Patient- 633342  done!
processing:  652133


/home/zhenwei/anaconda3/envs/mma/lib/python3.8/site-packages/sklearn/mixture/_base.py:143: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/zhenwei/anaconda3/envs/mma/lib/python3.8/site-packages/sklearn/mixture/_base.py:143: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/zhenwei/anaconda3/envs/mma/lib/python3.8/site-packages/sklearn/mixture/_base.py:143: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(


Patient- 652133  done!
processing:  689326
Patient- 689326  done!
processing:  687346
Patient- 687346  done!
processing:  645654
Patient- 645654  done!
processing:  689262
Patient- 689262  done!
processing:  646021
Patient- 646021  done!
processing:  674632
Patient- 674632  done!
processing:  664067
Patient- 664067  done!
processing:  662924
Patient- 662924  done!
processing:  683166
Patient- 683166  done!
processing:  664066
Patient- 664066  done!
processing:  636017
Patient- 636017  done!
processing:  640531
Patient- 640531  done!
processing:  645010
Patient- 645010  done!
processing:  652618
Patient- 652618  done!
processing:  676966
Patient- 676966  done!
processing:  644164
Patient- 644164  done!
processing:  647620
Patient- 647620  done!
processing:  671547
Patient- 671547  done!
processing:  654382
Patient- 654382  done!
processing:  662403
Patient- 662403  done!
processing:  686476
Patient- 686476  done!
processing:  652736
Patient- 652736  done!
processing:  647659
Patient- 64

#### Generate full ED table with clinical outcome

In [40]:
c_path = '/home/zhenwei/Documents/ProjectCode/image_processing/data/'
EDI_path = '/home/zhenwei/Documents/ProjectCode/image_processing/SVoxel/output_HN/EDI'

df_c = pd.read_csv(os.path.join(c_path,center + '_ClinicalData.csv'))
df_BSV = pd.read_csv(os.path.join(EDI_path, center + '_BSV_EDI_k10.csv'))

df_merge = df_c.merge(df_BSV,on='ID')
df_merge.to_csv(os.path.join(EDI_path,center + '_BSV_EDI__k10_full.csv'),index=False)